# County Features 

We tried using the ACS 1 year data through Cenpy and the API and consistently got ~820 to 840 counties. 

I tried it from the API directly and from cenpy, with different years for each, and it always returned counties within that range. And, not all states had counties in there, so it wasn't a random sample (remember, Arizona had zero...) 

#### However, using the 5 year data does get us more county information! 

In [1]:
import pandas as pd
import cenpy as c
#find table that we want to query 
available = c.explorer.available()
acs_df = available[available['title'].str.contains('ACS') == True]
acs_df = acs_df[acs_df['vintage'] == 2017]
acs_df

/Users/tjjj/anaconda3/lib/python3.7/site-packages/libpysal/weights/util.py:19: UserWarning: geopandas not available. Some functionality will be disabled.
  warn('geopandas not available. Some functionality will be disabled.')


,variable_cell_limit,title,temporal,spatial,publisher,programCode,modified,keyword,distribution,description,...,c_isCube,c_isAvailable,c_isAggregate,c_groupsLink,c_geographyLink,c_examplesLink,c_dataset,c.is_cell_limit_discovery,bureauCode,accessLevel
ACSDT1Y2017,NaN,ACS 1-Year Detailed Tables,unidentified,NaN,U.S. Census Bureau,006:004,2018-09-13 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs1/grou...,https://api.census.gov/data/2017/acs/acs1/geog...,https://api.census.gov/data/2017/acs/acs1/exam...,"(acs, acs1)",NaN,NaN,NaN
ACSCP1Y2017,NaN,ACS 1-Year Comparison Profiles,unidentified,NaN,U.S. Census Bureau,006:004,2018-09-13 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs1/cpro...,https://api.census.gov/data/2017/acs/acs1/cpro...,https://api.census.gov/data/2017/acs/acs1/cpro...,"(acs, acs1, cprofile)",NaN,NaN,NaN
ACSDP1Y2017,NaN,ACS 1-Year Data Profiles,unidentified,NaN,U.S. Census Bureau,006:004,2018-09-13 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs1/prof...,https://api.census.gov/data/2017/acs/acs1/prof...,https://api.census.gov/data/2017/acs/acs1/prof...,"(acs, acs1, profile)",NaN,NaN,NaN
ACSSPP1Y2017,NaN,ACS 1-Year Selected Population Profiles,unidentified,NaN,U.S. Census Bureau,006:004,2018-09-17 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",Selected Population Profiles provide broad soc...,...,True,True,True,https://api.census.gov/data/2017/acs/acs1/spp/...,https://api.census.gov/data/2017/acs/acs1/spp/...,https://api.census.gov/data/2017/acs/acs1/spp/...,"(acs, acs1, spp)",NaN,NaN,NaN
ACSST1Y2017,NaN,ACS 1-Year Subject Tables,unidentified,NaN,U.S. Census Bureau,006:004,2018-09-13 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs1/subj...,https://api.census.gov/data/2017/acs/acs1/subj...,https://api.census.gov/data/2017/acs/acs1/subj...,"(acs, acs1, subject)",NaN,NaN,NaN
ACSDT5Y2017,NaN,ACS 5-Year Detailed Tables,unidentified,NaN,U.S. Census Bureau,006:004,2018-08-21 07:11:43.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs5/grou...,https://api.census.gov/data/2017/acs/acs5/geog...,https://api.census.gov/data/2017/acs/acs5/exam...,"(acs, acs5)",NaN,NaN,NaN
ACSCP5Y2017,NaN,ACS 5-Year Comparison Profiles,unidentified,NaN,U.S. Census Bureau,006:004,2018-10-19 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs5/cpro...,https://api.census.gov/data/2017/acs/acs5/cpro...,https://api.census.gov/data/2017/acs/acs5/cpro...,"(acs, acs5, cprofile)",NaN,NaN,NaN
ACSDP5Y2017,NaN,ACS 5-Year Data Profiles,unidentified,NaN,U.S. Census Bureau,006:004,2018-10-19 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs5/prof...,https://api.census.gov/data/2017/acs/acs5/prof...,https://api.census.gov/data/2017/acs/acs5/prof...,"(acs, acs5, profile)",NaN,NaN,NaN
ACSST5Y2017,NaN,ACS 5-Year Subject Tables,unidentified,NaN,U.S. Census Bureau,006:004,2018-10-19 00:00:00.0,(),"{'@type': 'dcat:Distribution', 'accessURL': 'h...",The American Community Survey (ACS) is an ongo...,...,True,True,True,https://api.census.gov/data/2017/acs/acs5/subj...,https://api.census.gov/data/2017/acs/acs5/subj...,https://api.census.gov/data/2017/acs/acs5/subj...,"(acs, acs5, subject)",Na

Based on this website and other research, we want to use ACSDP5Y2017.

In [2]:
c.explorer.explain('ACSDP5Y2017')

{'ACS 5-Year Data Profiles': 'The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services. The ACS covers a broad range of topics about social, economic, demographic, and housing characteristics of the U.S. population. The data profiles include the following geographies: nation, all states (including DC and Puerto Rico), all metropolitan areas, all congressional districts, all counties, all places and all tracts. Data profiles contain broad social, economic, housing, and demographic information. The data are presented as both counts and percentages. There are over 2,400 variables in this dataset.'}

In [3]:
con = c.base.Connection('ACSDP5Y2017')
g_unit = 'county:*'

In [4]:
#create education features list from https://api.census.gov/data/2017/acs/acs5/profile/variables.html
cols_edu = [] 
for n in range(52,68): 
    var_name = 'DP02_00'+str(n)+'PE'
    cols_edu.append(var_name)

#create internet features list
cols_internet = []
for n in range(150,153):
    var_name = 'DP02_0'+str(n)+'PE'
    cols_internet.append(var_name)

#create row names for joining and EDA
cols_req = ['NAME']


In [5]:
cols = cols_req + cols_edu + cols_internet 

In [6]:
counties_df = con.query(cols=cols, geo_unit=g_unit)
counties_df.head()

,NAME,DP02_0052PE,DP02_0053PE,DP02_0054PE,DP02_0055PE,DP02_0056PE,DP02_0057PE,DP02_0058PE,DP02_0059PE,DP02_0060PE,...,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,state,county
0,"Pickens County, Alabama",4416,11.8,5.7,39.2,26.8,16.5,14241,6.2,13.9,...,7.5,8.8,3.0,79.8,11.8,7620,71.0,60.9,01,107
1,"Sumter County, Alabama",4106,4.4,5.9,26.9,18.8,44.1,8244,4.7,12.6,...,7.0,10.6,7.6,82.7,18.2,5073,64.8,50.4,01,119
2,"Jefferson County, Alabama",165739,6.8,5.7,39.9,20.5,27.1,447048,3.0,7.6,...,8.1,19.4,12.5,89.4,31.9,261390,84.4,73.0,01,073
3,"Choctaw County, Alabama",2718,3.9,4.3,48.0,24.4,19.4,9449,6.6,13.3,...,9.1,7.9,3.8,80.1,11.6,5463,70.4,52.3,01,023
4,"Franklin County, Alabama",7426,3.3,6.8,49.2,23.3,17.4,20734,11.8,11.9,...,7.5,8.3,5.1,76.4,13.4,11533,74.2,60.3,01,059


In [7]:
counties_df = counties_df.rename(index=str, columns={'NAME': 'county_name', 
                                       'DP02_0052PE': '%_inschool_3+',
                                       'DP02_0053PE': '%_preschool_3+',
                                       'DP02_0054PE': '%_kinderg_3+',
                                       'DP02_0055PE': '%_elementary_3+',
                                       'DP02_0056PE': '%_highschool_3+',
                                       'DP02_0057PE': '%_college_3+',
                                       'DP02_0058PE': '%_25+',
                                       'DP02_0059PE': '%_below9th_25+',
                                       'DP02_0060PE': '%_9th-12th_25+',
                                       'DP02_0061PE': '%_hsgrad_25+',
                                       'DP02_0062PE': '%_somecollege_25+',
                                       'DP02_0063PE': '%_associates_25+',
                                       'DP02_0064PE': '%_bachelors_b25+',
                                       'DP02_0065PE': '%_gradschool_25+',
                                       'DP02_0066PE': '%_hsgrad_or+_25+',
                                       'DP02_0067PE': '%_bachelors_or+_25+',
                                       'DP02_0150PE': '%_useinternet_total_households',
                                       'DP02_0151PE': '%_havecomp_total_households',
                                       'DP02_0152PE': '%_broadband_total_households',
                                      })

In [8]:
counties_df[counties_df['county_name'].str.contains('Wyoming')]

,county_name,%_inschool_3+,%_preschool_3+,%_kinderg_3+,%_elementary_3+,%_highschool_3+,%_college_3+,%_25+,%_below9th_25+,%_9th-12th_25+,...,%_associates_25+,%_bachelors_b25+,%_gradschool_25+,%_hsgrad_or+_25+,%_bachelors_or+_25+,%_useinternet_total_households,%_havecomp_total_households,%_broadband_total_households,state,county
1857,"Wyoming County, New York",7920,5.7,4.3,46.4,25.3,18.3,29320,3.5,8.2,...,13.4,9.7,5.8,88.3,15.4,15686,83.8,73.1,36,121
2267,"Wyoming County, Pennsylvania",5697,5.8,4.8,42.4,25.4,21.6,19657,1.8,6.1,...,8.5,12.4,6.5,92.1,18.8,10801,84.7,78.6,42,131
2966,"Wyoming County, West Virginia",4207,6.6,7.6,49.3,24.6,11.9,15859,8.5,13.1,...,4.1,5.4,3.9,78.4,9.3,9169,71.8,63.6,54,109
3047,"Goshen County, Wyoming",2938,5.4,3.2,42.1,24.1,25.3,9449,2.0,6.6,...,11.0,15.3,9.1,91.4,24.4,5328,81.9,71.3,56,015
3048,"Uinta County, Wyoming",5557,6.8,6.3,53.9,21.6,11.5,12978,2.3,5.9,...,10.3,11.9,5.5,91.8,17.4,7705,91.2,83.8,56,041
3049,"Washakie County, Wyoming",1783,4.7,2.5,52.7,33.5,6.7,5705,2.9,8.6,...,11.3,15.4,5.6,88.5,21.0,3490,87.7,76.4,56,043
3050,"Hot Springs County, Wyoming",943,10.0,6.0,49.9,19.3,14.7,3604,1.2,6.2,...,14.1,13.3,8.3,92.5,21.6,2246,85.1,74.4,56,017
3051,"Fremont County, Wyoming",10169,9.1,5.7,45.1,21.1,19.0,26781,2.1,6.8,...,10.6,15.5,7.7,91.1,23.3,15167,86.1,71.9,56,013
3052,"Sublette County, Wyoming",2482,6.8,7.7,49.4,21.5,14.6,7051,1.6,2.1,...,9.2,19.2,6.2,96.2,25.4,3197,93.9,82.9,56,035
3053,"Weston County, Wyoming",1441,13.4,2.7,46.2,19.4,18.3,5074,1.5,6.6,...,9.1,14.1,5.7,91.9,19.8,3182,82.9,71.1,56,045


In [9]:
counties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3220 entries, 0 to 3219
Data columns (total 22 columns):
county_name                       3220 non-null object
%_inschool_3+                     3142 non-null object
%_preschool_3+                    3142 non-null object
%_kinderg_3+                      3142 non-null object
%_elementary_3+                   3142 non-null object
%_highschool_3+                   3142 non-null object
%_college_3+                      3142 non-null object
%_25+                             3142 non-null object
%_below9th_25+                    3142 non-null object
%_9th-12th_25+                    3142 non-null object
%_hsgrad_25+                      3142 non-null object
%_somecollege_25+                 3142 non-null object
%_associates_25+                  3142 non-null object
%_bachelors_b25+                  3142 non-null object
%_gradschool_25+                  3142 non-null object
%_hsgrad_or+_25+                  3142 non-null object
%_bachelors_or+_25

In [10]:
counties_df['county_code'] = counties_df['state'] + counties_df['county']

counties_df.head()

,county_name,%_inschool_3+,%_preschool_3+,%_kinderg_3+,%_elementary_3+,%_highschool_3+,%_college_3+,%_25+,%_below9th_25+,%_9th-12th_25+,...,%_bachelors_b25+,%_gradschool_25+,%_hsgrad_or+_25+,%_bachelors_or+_25+,%_useinternet_total_households,%_havecomp_total_households,%_broadband_total_households,state,county,county_code
0,"Pickens County, Alabama",4416,11.8,5.7,39.2,26.8,16.5,14241,6.2,13.9,...,8.8,3.0,79.8,11.8,7620,71.0,60.9,01,107,01107
1,"Sumter County, Alabama",4106,4.4,5.9,26.9,18.8,44.1,8244,4.7,12.6,...,10.6,7.6,82.7,18.2,5073,64.8,50.4,01,119,01119
2,"Jefferson County, Alabama",165739,6.8,5.7,39.9,20.5,27.1,447048,3.0,7.6,...,19.4,12.5,89.4,31.9,261390,84.4,73.0,01,073,01073
3,"Choctaw County, Alabama",2718,3.9,4.3,48.0,24.4,19.4,9449,6.6,13.3,...,7.9,3.8,80.1,11.6,5463,70.4,52.3,01,023,01023
4,"Franklin County, Alabama",7426,3.3,6.8,49.2,23.3,17.4,20734,11.8,11.9,...,8.3,5.1,76.4,13.4,11533,74.2,60.3,01,059,01059


In [11]:
counties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3220 entries, 0 to 3219
Data columns (total 23 columns):
county_name                       3220 non-null object
%_inschool_3+                     3142 non-null object
%_preschool_3+                    3142 non-null object
%_kinderg_3+                      3142 non-null object
%_elementary_3+                   3142 non-null object
%_highschool_3+                   3142 non-null object
%_college_3+                      3142 non-null object
%_25+                             3142 non-null object
%_below9th_25+                    3142 non-null object
%_9th-12th_25+                    3142 non-null object
%_hsgrad_25+                      3142 non-null object
%_somecollege_25+                 3142 non-null object
%_associates_25+                  3142 non-null object
%_bachelors_b25+                  3142 non-null object
%_gradschool_25+                  3142 non-null object
%_hsgrad_or+_25+                  3142 non-null object
%_bachelors_or+_25

In [12]:
counties_df['county_code'] = counties_df['county_code'].astype(int)
counties_df['state'] = counties_df['state'].astype(int)
counties_df['county'] = counties_df['county'].astype(int)
counties_df.iloc[:,1:-3] = counties_df.iloc[:,1:-3].astype(float)
counties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3220 entries, 0 to 3219
Data columns (total 23 columns):
county_name                       3220 non-null object
%_inschool_3+                     3142 non-null float64
%_preschool_3+                    3142 non-null float64
%_kinderg_3+                      3142 non-null float64
%_elementary_3+                   3142 non-null float64
%_highschool_3+                   3142 non-null float64
%_college_3+                      3142 non-null float64
%_25+                             3142 non-null float64
%_below9th_25+                    3142 non-null float64
%_9th-12th_25+                    3142 non-null float64
%_hsgrad_25+                      3142 non-null float64
%_somecollege_25+                 3142 non-null float64
%_associates_25+                  3142 non-null float64
%_bachelors_b25+                  3142 non-null float64
%_gradschool_25+                  3142 non-null float64
%_hsgrad_or+_25+                  3142 non-null float64
%_b

In [13]:
counties_df[pd.isnull(counties_df).any(axis=1)]

,county_name,%_inschool_3+,%_preschool_3+,%_kinderg_3+,%_elementary_3+,%_highschool_3+,%_college_3+,%_25+,%_below9th_25+,%_9th-12th_25+,...,%_bachelors_b25+,%_gradschool_25+,%_hsgrad_or+_25+,%_bachelors_or+_25+,%_useinternet_total_households,%_havecomp_total_households,%_broadband_total_households,state,county,county_code
3142,"Jayuya Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,73,72073
3143,"Quebradillas Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,115,72115
3144,"Guayama Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,57,72057
3145,"Guánica Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,55,72055
3146,"Rincón Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,117,72117
3147,"Villalba Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,149,72149
3148,"Aguas Buenas Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,7,72007
3149,"Bayamón Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,21,72021
3150,"Hormigueros Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,67,72067
3151,"Manatí Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,91,72091


In [14]:
counties_df = counties_df.dropna()
counties_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3142 entries, 0 to 3141
Data columns (total 23 columns):
county_name                       3142 non-null object
%_inschool_3+                     3142 non-null float64
%_preschool_3+                    3142 non-null float64
%_kinderg_3+                      3142 non-null float64
%_elementary_3+                   3142 non-null float64
%_highschool_3+                   3142 non-null float64
%_college_3+                      3142 non-null float64
%_25+                             3142 non-null float64
%_below9th_25+                    3142 non-null float64
%_9th-12th_25+                    3142 non-null float64
%_hsgrad_25+                      3142 non-null float64
%_somecollege_25+                 3142 non-null float64
%_associates_25+                  3142 non-null float64
%_bachelors_b25+                  3142 non-null float64
%_gradschool_25+                  3142 non-null float64
%_hsgrad_or+_25+                  3142 non-null float64
%_b

In [15]:
counties_df.tail().T

,3137,3138,3139,3140,3141
county_name,"Rusk County, Wisconsin","Vilas County, Wisconsin","Shawano County, Wisconsin","Juneau County, Wisconsin","Washington County, Wisconsin"
%_inschool_3+,2690,3495,8769,5183,31410
%_preschool_3+,6.4,6.7,6.5,4.8,5.8
%_kinderg_3+,3.8,5.8,5.7,7.2,5.5
%_elementary_3+,53.5,46.2,46.4,46.5,44.6
%_highschool_3+,25.1,23.1,24.3,25.9,24.4
%_college_3+,11.1,18.1,17,15.6,19.8
%_25+,10268,16692,29299,19276,93734
%_below9th_25+,4.8,1,2.5,3.6,1.8
%_9th-12th_25+,8.2,6.4,6.5,9.4,3.8


# Overdoses by County

In [16]:
od = pd.read_csv('2013-2017 Overdose Information.txt', delimiter='\t')
od = od.drop(['Notes'], axis=1)

od_reliable = od.drop(od[od['Crude Rate']=='Suppressed'].index)

od_reliable = od_reliable.drop(od_reliable[od_reliable['Crude Rate']=='Unreliable'].index)
od_reliable = od_reliable.drop(od_reliable[od_reliable['Deaths']=='Missing'].index)
od_reliable = od_reliable.drop(od_reliable[od_reliable['Deaths'].isna()].index)
od_reliable = od_reliable.drop(od_reliable[od_reliable['County Code'].isna()].index)
od_reliable.head()

,County,County Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001.0,47,276908,16.9731,0.0103%
1,"Baldwin County, AL",1003.0,253,1020551,24.7905,0.0554%
3,"Bibb County, AL",1007.0,27,112912,23.9124,0.0059%
4,"Blount County, AL",1009.0,88,288981,30.4518,0.0193%
7,"Calhoun County, AL",1015.0,132,577611,22.8528,0.0289%


In [17]:
od_reliable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 3144
Data columns (total 6 columns):
County               2016 non-null object
County Code          2016 non-null float64
Deaths               2016 non-null object
Population           2016 non-null object
Crude Rate           2016 non-null object
% of Total Deaths    2016 non-null object
dtypes: float64(1), object(5)
memory usage: 110.2+ KB


In [18]:
od_reliable['County Code'] = od_reliable['County Code'].astype(int)
od_reliable['Deaths'] = od_reliable['Deaths'].astype(int)
od_reliable['Population'] = od_reliable['Population'].astype(int)
od_reliable['Crude Rate'] = od_reliable['Crude Rate'].astype(float)

In [19]:
od_reliable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 3144
Data columns (total 6 columns):
County               2016 non-null object
County Code          2016 non-null int64
Deaths               2016 non-null int64
Population           2016 non-null int64
Crude Rate           2016 non-null float64
% of Total Deaths    2016 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 110.2+ KB


In [20]:
od_reliable['% of Total Deaths'] = od_reliable['% of Total Deaths'].str.replace("%", "")
od_reliable['% of Total Deaths'] = od_reliable['% of Total Deaths'].astype(float)

In [21]:
od_df = od_reliable

In [22]:
od_df.head()

,County,County Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001,47,276908,16.9731,0.0103
1,"Baldwin County, AL",1003,253,1020551,24.7905,0.0554
3,"Bibb County, AL",1007,27,112912,23.9124,0.0059
4,"Blount County, AL",1009,88,288981,30.4518,0.0193
7,"Calhoun County, AL",1015,132,577611,22.8528,0.0289


In [23]:
od_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 3144
Data columns (total 6 columns):
County               2016 non-null object
County Code          2016 non-null int64
Deaths               2016 non-null int64
Population           2016 non-null int64
Crude Rate           2016 non-null float64
% of Total Deaths    2016 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 110.2+ KB


# Joining the CDC and Census Data

In [25]:
counties_data = od_df.merge(counties_df, how='inner', left_on='County Code', right_on='county_code')

In [29]:
counties_data.drop(['County Code', 'county_name'], axis=1, inplace=True)

In [30]:
counties_data

,County,Deaths,Population,Crude Rate,% of Total Deaths,%_inschool_3+,%_preschool_3+,%_kinderg_3+,%_elementary_3+,%_highschool_3+,...,%_bachelors_b25+,%_gradschool_25+,%_hsgrad_or+_25+,%_bachelors_or+_25+,%_useinternet_total_households,%_havecomp_total_households,%_broadband_total_households,state,county,county_code
0,"Autauga County, AL",47,276908,16.9731,0.0103,13438.0,3.9,7.1,46.3,24.8,...,14.5,10.5,87.7,25.0,21054.0,86.2,76.6,1,1,1001
1,"Baldwin County, AL",253,1020551,24.7905,0.0554,44265.0,6.2,4.9,46.5,24.3,...,20.4,10.3,90.2,30.7,76133.0,86.9,74.5,1,3,1003
2,"Bibb County, AL",27,112912,23.9124,0.0059,4626.0,3.0,4.9,44.0,28.7,...,8.1,5.1,82.1,13.2,6916.0,74.8,62.0,1,7,1007
3,"Blount County, AL",88,288981,30.4518,0.0193,12428.0,3.9,4.4,49.3,25.9,...,8.7,4.4,79.8,13.1,20690.0,78.2,65.8,1,9,1009
4,"Calhoun County, AL",132,577611,22.8528,0.0289,27798.0,6.2,5.3,41.4,20.6,...,10.2,7.6,83.2,17.9,45099.0,82.9,71.0,1,15,1015
5,"Chambers County, AL",34,169917,20.0098,0.0074,7081.0,5.6,3.3,48.2,21.5,...,9.0,4.3,80.9,13.3,13694.0,72.7,62.8,1,17,1017
6,"Cherokee County, AL",40,129681,30.8449,0.0088,5109.0,3.8,4.9,47.8,27.6,...,6.6,5.9,79.5,12.5,10795.0,79.4,67.5,1,19,1019
7,"Chilton County, AL",42,219833,19.1054,0.0092,9659.0,4.9,7.2,48.6,24.0,...,9.5,5.6,81.8,15.1,16768.0,73.8,56.6,1,21,1021
8,"Coffee County, AL",39,256158,15.2250,0.0085,11864.0,4.5,6.3,45.0,25.9,...,14.7,8.0,85.4,22.7,19620.0,84.1,75.3,1,31,1031
9,"Colbert County, AL",39,272133,14.3312,0.0085,11445.0,3.9,4.5,46.8,24.6,...,11.5,7.3,83.9,18.8,21983.0,80.3,65.9,1,33,1033
